In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
ostro_dir = 'Osteosarcoma-UT'



# Define image dimensions and batch size
img_height = 224
img_width = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'
)


test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [9]:
from sklearn.model_selection import train_test_split
import os

# Define the root directory containing subfolders for each class
root_dir = 'Osteosarcoma-UT'

# Get the list of class names
classes = os.listdir(root_dir)
print(classes)

for i in classes:
    clas_path = os.path.join(root_dir,i)
    print("{} contains {} images".format(i,len(os.listdir(clas_path))))

# Split the data into training and validation sets
train_data = []
val_data = []
for cls in classes:
    cls_dir = os.path.join(root_dir, cls)
    images = [os.path.join(cls_dir, img) for img in os.listdir(cls_dir)]
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)
    train_data.extend([(img, cls) for img in train_images])
    val_data.extend([(img, cls) for img in val_images])

# Further split the training data into training and testing sets
train_images, test_images = train_test_split(train_data, test_size=0.2, random_state=42)

# Now you have train_images, val_data, and test_images containing paths to images for each set



['Viable', 'Non-Tumor', 'Non-Viable-Tumor']
Viable contains 295 images
Non-Tumor contains 533 images
Non-Viable-Tumor contains 316 images


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame with two columns: 'file_path' and 'label'
train_df = pd.DataFrame(train_data, columns=['file_path', 'label'])
val_df = pd.DataFrame(val_data, columns=['file_path', 'label'])
test_df = pd.DataFrame(test_images, columns=['file_path', 'label'])

# Split the data into file paths and labels
train_data = train_df['file_path']
train_labels = train_df['label']

val_data = val_df['file_path']
val_labels = val_df['label']

test_data = test_df['file_path']
test_labels = test_df['label']

# You can create separate data generators for each set
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 914 validated image filenames belonging to 3 classes.
Found 230 validated image filenames belonging to 3 classes.
Found 183 validated image filenames belonging to 3 classes.


In [11]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [12]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
def create_cnn(input_shape=(224, 224, 3), num_classes=3):  # Assuming 3 classes: non tumor, non viable tumor, viable tumor
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Create CNN model
model = create_cnn()

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Change loss function accordingly
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch= len(train_generator),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator),
                    callbacks=[early_stopping])




2024-02-22 10:17:12.487469: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-22 10:17:12.487613: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-22 10:17:12.487627: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-22 10:17:12.488376: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-22 10:17:12.488622: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

2024-02-22 10:17:13.839948: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


29/29 [==============================] - 20s 535ms/step - loss: 1.3406 - accuracy: 0.4344 - val_loss: 1.0613 - val_accuracy: 0.4652
Epoch 2/100
29/29 [==============================] - 15s 517ms/step - loss: 1.0191 - accuracy: 0.4748 - val_loss: 0.9254 - val_accuracy: 0.4739
Epoch 3/100
29/29 [==============================] - 15s 523ms/step - loss: 0.9696 - accuracy: 0.4770 - val_loss: 0.8945 - val_accuracy: 0.5609
Epoch 4/100
29/29 [==============================] - 15s 516ms/step - loss: 0.9183 - accuracy: 0.5875 - val_loss: 0.7957 - val_accuracy: 0.6348
Epoch 5/100
29/29 [==============================] - 15s 515ms/step - loss: 0.8840 - accuracy: 0.6182 - val_loss: 0.9469 - val_accuracy: 0.6043
Epoch 6/100
29/29 [==============================] - 15s 516ms/step - loss: 0.8023 - accuracy: 0.6565 - val_loss: 0.8700 - val_accuracy: 0.7261
Epoch 7/100
29/29 [==============================] - 15s 518ms/step - loss: 0.9502 - accuracy: 0.6138 - val_loss: 1.0866 - val_accuracy: 0.4565
Epoc

In [14]:
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score

# Assuming you already have your test generator and model trained

# Evaluate the model using test generator
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test accuracy:", test_acc)

# Get predictions and true labels
# Ensure that steps is set to get the correct number of samples
y_pred = model.predict_generator(test_generator, steps=test_generator.samples // batch_size + 1)
y_true = test_generator.classes

# Only take the necessary number of samples from y_true
y_true = y_true[:len(y_pred)]

# Calculate macro-averaged F1-score
macro_f1 = f1_score(y_true, y_pred.argmax(axis=1), average='macro')
print("Macro-averaged F1-score:", macro_f1)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred.argmax(axis=1))
print("Confusion matrix:\n", cm)

# Calculate AUC-ROC score for each class
y_onehot = np.zeros((len(y_true), len(np.unique(y_true))))
y_onehot[np.arange(len(y_true)), y_true] = 1
auc_roc_scores = roc_auc_score(y_onehot, y_pred, multi_class='ovr')
print("AUC-ROC scores for each class:", auc_roc_scores)

# Calculate average AUC-ROC score
average_auc_roc = np.mean(auc_roc_scores)
print("Average AUC-ROC score:", average_auc_roc)


5/5 [==============================] - 1s 252ms/step - loss: 0.5279 - accuracy: 0.7875
Test accuracy: 0.7875000238418579


/var/folders/yd/6l3qbbqj2qx_y_lxrr45xd480000gn/T/ipykernel_18848/3608557048.py:12: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = model.predict_generator(test_generator, steps=test_generator.samples // batch_size + 1)


Macro-averaged F1-score: 0.7427622211675368
Confusion matrix:
 [[87  3  3]
 [ 6 30  8]
 [10  9 27]]
AUC-ROC scores for each class: 0.9272984973406134
Average AUC-ROC score: 0.9272984973406134
